In [1]:
import os
import numpy as np
import pandas as pd
import time
from selenium import webdriver
import csv
from selenium.webdriver.common.keys import Keys

In [2]:
# driverで開きたいURLを入力, 3行目でダウンロードしたchromedriverのあるパスを指定
url = "https://boxil.jp/categories/"
driver = webdriver.Chrome(executable_path='/Users/hiroki_igeta/chromedriver')
driver.get(url)
driver.maximize_window()

In [33]:
#各カテゴリ一覧のURLを取得する
def scrape_url(driver):
    l = []
    for i in driver.find_elements_by_class_name('pagesCategories-content-childCategory'):
        for url in i.find_elements_by_tag_name('a'):
            l.append(url.get_attribute('href'))
    return pd.Series(data = l)

In [10]:
results =  scrape_url(driver)
#csvファイルにカテゴリーごとのURLを保存する
results.to_csv("data/categories_url.csv", header=False, index=False)

In [3]:
#カテゴリーページで次のページに飛ぶ
def next_page(driver, urls):
    try:
        ul_pagination = driver.find_element_by_class_name('pagination-boxil')
        li_next = ul_pagination.find_element_by_class_name('next')
        driver.execute_script("window.scrollTo(0, 3900);")
#         print("did")
        time.sleep(2)
        li_next.find_element_by_tag_name('a').click()
        time.sleep(3)
        return True
    except:
        return False

In [4]:
# 現在表示されているページのサービス名を取得
def scrape_service(driver):
    # ここにスクレイピングデータを１行ずつ格納していく
    results = []
    urls = []
    is_on = True
#     print("-----------")
#     count = 1
    while is_on:
#         print(count)
#         count += 1
        urls.append(driver.current_url)
        for i in driver.find_elements_by_class_name("heading-3"):
#             l = []
            a = i.find_element_by_tag_name("a")
#             l.append(a.text)
#             l.append(a.get_attribute("href"))
#             results.append(l)
            results.append(a.get_attribute("href"))

        is_on = next_page(driver, urls)
    return results

In [5]:
# results = scrape_service(driver)
# print(results)

In [2]:
url = "https://boxil.jp/categories/"
driver = webdriver.Chrome(executable_path='/Users/hiroki_igeta/chromedriver')
driver.get(url)
driver.maximize_window()
driver.get(url)

NameError: name 'webdriver' is not defined

In [5]:
#各カテゴリーのURLを開き、それぞれのサービス名を取得する
with open('data/categories_url.csv', encoding='utf-8') as csv_file:
    service_data = []
    o_csv = []
    for row in csv.reader(csv_file):
        o_csv.append(row[0])
        for url in o_csv:
            driver.get(url)
            time.sleep(3)
            service_data.append(scrape_service(driver))
        del o_csv[0]
    tmp = pd.Series(data = service_data)
    tmp.to_csv("data/service_detail_url.csv", header=False, index=False)

In [159]:
#データ整形
df = pd.read_csv("data/service_names.csv", header=None)
result = []
df[0] = df[0].str.replace('[', '')
df[0] = df[0].str.replace(']', '')
df[0] = df[0].str.replace("'", "")
df[0] = df[0].str.replace("'", "")
df[0] = df[0].str.replace(" ", "")
df[0] = df[0].str.replace("　", "")
# バックオフィス「ペーパーレス化」完全ガイドbyBOXILSaaS
# インサイドセールス業界レポート2019byスマートキャンプ
# インサイドセールス業界レポート2020byスマートキャンプ
# SaaS業界レポート2020
# SaaS業界レポート2021速報版

for d in df[0]:
    for dd in d.split(","):
        result.append(dd)

# result = pd.Series(data = list(set(result)))
# result = pd.Series(data = result)
result = pd.Series(data = list(dict.fromkeys(result)))
result.to_csv("data/service_names_lists.csv", header=False, index=False)

In [28]:
df = pd.read_csv("data/service_detail_url.csv", header=None)
result = []
df[0] = df[0].str.replace('[', '')
df[0] = df[0].str.replace(']', '')
df[0] = df[0].str.replace("'", "")
df[0] = df[0].str.replace("'", "")
df[0] = df[0].str.replace(" ", "")
df[0] = df[0].str.replace("　", "")
# バックオフィス「ペーパーレス化」完全ガイドbyBOXILSaaS
# インサイドセールス業界レポート2019byスマートキャンプ
# インサイドセールス業界レポート2020byスマートキャンプ
# SaaS業界レポート2020
# SaaS業界レポート2021速報版

for d in df[0]:
    for dd in d.split(","):
        result.append(dd)

# result = pd.Series(data = list(set(result)))
# result = pd.Series(data = result)
result = pd.Series(data = list(dict.fromkeys(result)))
result.to_csv("data/service_detail_url_lists.csv", header=False, index=False)

In [3]:
#サービス名・企業名・資本金・従業員規模等を取得するコード
def scrape_detail(driver):
    with open('data/detail_data.csv', 'a') as f:
        writer = csv.writer(f)
        corporate_detail = []
        try:
            corporate_detail.append(driver.find_element_by_class_name('service_show_overview__header__main__content__header-title').text) #サービス名
        except:
            corporate_detail.append("")
        try:
            detail = driver.find_element_by_id('ss-corporation')
            try:
                corporate_detail.append(detail.find_element_by_class_name('corporationOverviewWrapper__info-name').text) #会社名
            except:
                corporate_detail.append("")
                
            try:
                corporate_detail.append(detail.find_elements_by_class_name('corporationOverviewWrapper__info-detail')[0].text) #業種
            except:
                corporate_detail.append("")
                
            try:
                corporate_detail.append(detail.find_elements_by_class_name('corporationOverviewWrapper__info-detail')[1].text) #住所
            except:
                corporate_detail.append("")
                
            try:
                corporate_detail.append(detail.find_elements_by_class_name('corporationDetailRowWrapper__texts-text')[0].text) #資本金
            except:
                corporate_detail.append("")
                
            try:
                corporate_detail.append(detail.find_elements_by_class_name('corporationDetailRowWrapper__texts-text')[1].text) #代表者名
            except:
                corporate_detail.append("")
                
            try:
                corporate_detail.append(detail.find_elements_by_class_name('corporationDetailRowWrapper__texts-text')[2].text) #従業員規模
            except:
                corporate_detail.append("")
                
            try:
                corporate_detail.append(detail.find_elements_by_class_name('corporationDetailRowWrapper__texts-text')[3].text) # 企業URL
            except:
                corporate_detail.append("")
                
            try:
                corporate_detail.append(detail.find_elements_by_class_name('corporationDetailRowWrapper__texts-text')[4].text) #設立年月
            except:
                corporate_detail.append("")
                
        except:
            for _ in range(8):
                corporate_detail.append("")
        try:
            owned_service = detail.find_elements_by_class_name('corporationDetailWrapper__ownedService') #その他サービス名（リスト型）
            tmp = []
            for s in owned_service:
                tmp.append(s.text)
            corporate_detail.append(tmp)
        except:
            corporate_detail.append("")
        writer.writerow(corporate_detail)
        return corporate_detail

In [4]:
# results = []
# results.append(scrape_detail(driver))
# print(results)

In [4]:
#各カテゴリーのURLを開き、それぞれのサービス名・企業名等を取得する
with open('data/url_tmp.csv', encoding='utf-8') as csv_file:
    results = []
    o_csv = []
    for row in csv.reader(csv_file):
        o_csv.append(row[0])
        for url in o_csv:
            driver.get(url)
            time.sleep(3)
            corporate_detail = scrape_detail(driver)
            results.append(corporate_detail)
        del o_csv[0]
    tmp = pd.Series(data = results)
    tmp.to_csv("data/detail_data_complete.csv", header=False, index=False)

KeyboardInterrupt: 

In [ ]:
#114行目のBizPay改善する